# NLP Tasks (Part 2)

Question answering task

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 30/12/2025   | Martin | Created   | Notebook for Question-answering task | 

# Content

* [Introduction](#introduction)

# Introduction

Extracative Question Answering: Finding the answer of a question from provided text

- Dataset: Question and Answers from `SQuAD`
- Training only has a single answer per question, but evaluation has multiple answers
- Model is trained to predict the __start and end__ logit per token in the input context

In [34]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [35]:
raw_dataset = load_dataset("squad")
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [36]:
print("Context: ", raw_dataset["train"][0]["context"])
print("Question: ", raw_dataset["train"][0]["question"])
print("Answer: ", raw_dataset["train"][0]["answers"])

Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


Define tokenizer. Sentences will be in the format:

> [CLS] question [SEP] context

Truncate the context to limit each data point length. Will create multiple samples from a single question-answer pair. Some tokens will overflow into the next example.

- Will create additional entries for those that overflow
- For those questions where the answer is not contained, predict 0 for start and end logits

In [37]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.is_fast

True

In [38]:
context = raw_dataset["train"][0]["context"]
question = raw_dataset["train"][0]["question"]

inputs = tokenizer(question, context)
print(tokenizer.decode(inputs["input_ids"]))

[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building ' s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]


In [39]:
inputs = tokenizer(
  question,
  context,
  max_length=100,                 # Maximum length of context
  truncation='only_second',       # Context is in the second position of example
  stride=50,                      # Number of tokens to overlap from previous truncation
  return_overflowing_tokens=True, # Indicate to take overflowing tokens
  return_offsets_mapping=True
)

for ids in inputs['input_ids']:
  print(tokenizer.decode(ids))

[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building ' s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basi [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Next to the Main Building is the

In [40]:
inputs.keys()

KeysView({'input_ids': [[101, 1706, 2292, 1225, 1103, 6567, 2090, 9273, 2845, 1107, 8109, 1107, 10111, 20500, 1699, 136, 102, 22182, 1193, 117, 1103, 1278, 1144, 170, 2336, 1959, 119, 1335, 4184, 1103, 4304, 4334, 112, 188, 2284, 10945, 1110, 170, 5404, 5921, 1104, 1103, 6567, 2090, 119, 13301, 1107, 1524, 1104, 1103, 4304, 4334, 1105, 4749, 1122, 117, 1110, 170, 7335, 5921, 1104, 4028, 1114, 1739, 1146, 14089, 5591, 1114, 1103, 7051, 107, 159, 21462, 1566, 24930, 2508, 152, 1306, 3965, 107, 119, 5893, 1106, 1103, 4304, 4334, 1110, 1103, 19349, 1104, 1103, 11373, 4641, 119, 13301, 1481, 1103, 171, 17506, 102], [101, 1706, 2292, 1225, 1103, 6567, 2090, 9273, 2845, 1107, 8109, 1107, 10111, 20500, 1699, 136, 102, 1103, 4304, 4334, 1105, 4749, 1122, 117, 1110, 170, 7335, 5921, 1104, 4028, 1114, 1739, 1146, 14089, 5591, 1114, 1103, 7051, 107, 159, 21462, 1566, 24930, 2508, 152, 1306, 3965, 107, 119, 5893, 1106, 1103, 4304, 4334, 1110, 1103, 19349, 1104, 1103, 11373, 4641, 119, 13301, 1481, 

In [41]:
inputs = tokenizer(
  raw_dataset["train"][2:6]["question"],
  raw_dataset["train"][2:6]["context"],
  max_length=100,
  truncation="only_second",
  stride=50,
  return_overflowing_tokens=True,
  return_offsets_mapping=True,
)

In [42]:
inputs

{'input_ids': [[101, 1109, 19349, 1104, 1103, 11373, 1762, 1120, 10360, 8022, 1110, 3148, 1106, 1134, 2401, 136, 102, 22182, 1193, 117, 1103, 1278, 1144, 170, 2336, 1959, 119, 1335, 4184, 1103, 4304, 4334, 112, 188, 2284, 10945, 1110, 170, 5404, 5921, 1104, 1103, 6567, 2090, 119, 13301, 1107, 1524, 1104, 1103, 4304, 4334, 1105, 4749, 1122, 117, 1110, 170, 7335, 5921, 1104, 4028, 1114, 1739, 1146, 14089, 5591, 1114, 1103, 7051, 107, 159, 21462, 1566, 24930, 2508, 152, 1306, 3965, 107, 119, 5893, 1106, 1103, 4304, 4334, 1110, 1103, 19349, 1104, 1103, 11373, 4641, 119, 13301, 1481, 1103, 171, 17506, 102], [101, 1109, 19349, 1104, 1103, 11373, 1762, 1120, 10360, 8022, 1110, 3148, 1106, 1134, 2401, 136, 102, 1103, 4304, 4334, 1105, 4749, 1122, 117, 1110, 170, 7335, 5921, 1104, 4028, 1114, 1739, 1146, 14089, 5591, 1114, 1103, 7051, 107, 159, 21462, 1566, 24930, 2508, 152, 1306, 3965, 107, 119, 5893, 1106, 1103, 4304, 4334, 1110, 1103, 19349, 1104, 1103, 11373, 4641, 119, 13301, 1481, 1103, 1

In [43]:
for ids, mapping in zip(inputs['input_ids'], inputs['overflow_to_sample_mapping']):
  print(tokenizer.decode(ids))
  print(mapping)

[CLS] The Basilica of the Sacred heart at Notre Dame is beside to which structure? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building ' s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basi [SEP]
0
[CLS] The Basilica of the Sacred heart at Notre Dame is beside to which structure? [SEP] the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin [SEP]
0
[CLS] The Basilica of the Sacred heart at Notre Dame is beside to which structure? [SEP] Next to the M

Labels will be of the format: `(start_position, end_position)`

- Which are tuples of 2 integers representing the span of characters inside the original context
- If it appears before the context, then return `(0, 0)`
- Else loop to find the first and last token in the context

In [52]:
answers = raw_dataset['train'][2:6]['answers']
start_positions = []
end_positions = []

for i, offset in enumerate(inputs['offset_mapping']):
  answers_offset = inputs['overflow_to_sample_mapping'][i]
  answer = answers[answers_offset]
  start_char = answer['answer_start'][0]
  end_char = answer['answer_start'][0] + len(answer['text'][0])
  sequence_ids = inputs.sequence_ids(i)

  # Find the start and end of context
  cont_idx = 0
  while sequence_ids[cont_idx] != 1:
    cont_idx += 1
  start_cont_idx = cont_idx
  while sequence_ids[cont_idx] == 1:
    cont_idx += 1
  end_cont_idx = cont_idx - 1

  # Check if the answer is inside the context or not
  if offset[start_cont_idx][0] > start_char or offset[end_cont_idx][1] < end_char:
    start_positions.append(0)
    end_positions.append(0)
  else:
    idx = start_cont_idx
    while idx <= end_cont_idx and offset[idx][0] <= start_char:
      idx += 1
    start_positions.append(idx - 1)

    idx = end_cont_idx
    while idx >= start_cont_idx and offset[idx][1] >= end_char:
      idx -= 1
    end_positions.append(idx + 1)

start_positions, end_positions

([83, 51, 19, 0, 0, 64, 27, 0, 34, 0, 0, 0, 67, 34, 0, 0, 0, 0, 0],
 [85, 53, 21, 0, 0, 70, 33, 0, 40, 0, 0, 0, 68, 35, 0, 0, 0, 0, 0])

In [4]:
%load_ext watermark
%watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2025-12-30T18:04:06.937522+08:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.37.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.6.87.2-microsoft-standard-WSL2
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit

